In [2]:
!pip install gradio
!pip install torchscript

  Obtaining dependency information for gradio from https://files.pythonhosted.org/packages/d8/3c/fcfa9f2f9b97e9c38ccb30eb3edef8acbb284487bc7d8f5b98daf01cf757/gradio-4.12.0-py3-none-any.whl.metadata
  Using cached gradio-4.12.0-py3-none-any.whl.metadata (15 kB)
  Obtaining dependency information for altair<6.0,>=4.2.0 from https://files.pythonhosted.org/packages/c5/e4/7fcceef127badbb0d644d730d992410e4f3799b295c9964a172f92a469c7/altair-5.2.0-py3-none-any.whl.metadata
  Using cached altair-5.2.0-py3-none-any.whl.metadata (8.7 kB)
  Obtaining dependency information for fastapi from https://files.pythonhosted.org/packages/d4/e0/d5d6482e992a1892f3a9a62f6a9154944ae5b276e7da1cf92faa02e3a107/fastapi-0.108.0-py3-none-any.whl.metadata
  Using cached fastapi-0.108.0-py3-none-any.whl.metadata (24 kB)
  Using cached ffmpy-0.3.1.tar.gz (5.5 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  ERROR: Error [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software while executing command python setup.py egg_info
ERROR: Could not install packages due to an OSError: [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software

ERROR: Could not find a version that satisfies the requirement torchscript (from versions: none)
ERROR: No matching distribution found for torchscript


In [3]:
import torch
import torchvision.models as models


model = models.resnet18(pretrained=True)
model.eval()

# Create a sample input tensor (change according to your model's input requirements)
example_input = torch.randn(1, 3, 224, 224)

# Script the model
scripted_model = torch.jit.script(model)

# Save the scripted model to a file
scripted_model.save("scripted_resnet18.pt")

D:\anaconda\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
D:\anaconda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-dat

In [4]:
import torch
from PIL import Image
from io import BytesIO
import requests
import torchvision
from torchvision.models import  ResNet18_Weights
from torchvision import transforms



# Load the saved TorchScript model
model = torch.jit.load("scripted_resnet18.pt")


preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


image_url = 'https://images.unsplash.com/photo-1611267254323-4db7b39c732c?q=80&w=1000&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxzZWFyY2h8M3x8Y3V0ZSUyMGNhdHxlbnwwfHwwfHx8MA%3D%3D'
response = requests.get(image_url)
image = Image.open(BytesIO(response.content)).convert('RGB')


input_tensor = preprocess(image)
input_batch = input_tensor.unsqueeze(0)  # Add batch dimension


with torch.no_grad():
    # run the scripted model
    output = model(input_batch)

weights = ResNet18_Weights.DEFAULT

class_names = weights.meta["categories"]

# Get the top 5 predictions
probabilities = torch.nn.functional.softmax(output[0], dim=0)
top5_prob, top5_catid = torch.topk(probabilities, 5)

# Display top 5 predicted classes and their probabilities
for i in range(top5_prob.size(0)):
    class_idx = top5_catid[i].item()
    print(f"Prediction: {class_names[class_idx]}, Probability: {top5_prob[i].item():.4f}")

Prediction: tabby, Probability: 0.7669
Prediction: tiger cat, Probability: 0.1399
Prediction: Egyptian cat, Probability: 0.0867
Prediction: Persian cat, Probability: 0.0024
Prediction: lynx, Probability: 0.0012


In [7]:
import numpy as np
import gradio as gr

CLASSES = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

def softmax(x):
    return(np.exp(x - np.max(x)) / np.exp(x - np.max(x)).sum())


def classify_image(img):
    img = Image.fromarray(img.astype('uint8'), 'RGB')
    img = transforms.ToTensor()(img).unsqueeze(0)
    with torch.no_grad():
      prediction = torch.nn.functional.softmax(model(img)[0], dim=0)
    weights = ResNet18_Weights.DEFAULT
    class_names = weights.meta["categories"]
    confidences = {class_names[i]: float(prediction[i]) for i in range(len(class_names))}
    return confidences

ui = gr.Interface(fn=classify_image,
             inputs=gr.Image(),
             outputs=gr.Label(num_top_classes=3),
             # TODO replace example1.png example2.png with some images from your device
             examples=['example1.png', 'example2.png']
          )
ui.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
